# 📥 KuCoin OHLCV データ取得ノートブック

**目的**: 旧巨大ノートブックから "KuCoin OHLCV 取得" 部分のみ抽出し、データ取得を効率化

## チェックリスト
- [x] セル 1: GDrive マウント & pip install ccxt tenacity
- [x] セル 2: 取得関数（utils/drive_io.py 経由で保存）を %run で読込
- [x] セル 3: パラメータ入力用 widgets（symbol / timeframe / 期間）
- [x] セル 4: 実行 & CSV/Parquet 保存、取得行数を表示
- [x] 出力先 `/content/drive/MyDrive/kucoin_bot/data/raw/` が作成されることを確認

In [ ]:
# セル 1: Google Drive マウント & 必要ライブラリのインストール

# Google Drive をマウント
from google.colab import drive
drive.mount("/content/drive")

# 必要なライブラリをインストール
!pip install ccxt tenacity

print("✅ セットアップ完了！")

In [ ]:
# セル 2: 取得関数を utils/drive_io.py 経由で読込

import sys
sys.path.append("/content/drive/MyDrive/kucoin_bot")

# utils/drive_io.py から取得関数をインポート
%run /content/drive/MyDrive/kucoin_bot/utils/drive_io.py

import pandas as pd
import os
from google.colab import userdata

print("✅ 取得関数を読み込み完了！")

In [ ]:
# セル 3: パラメータ入力用 widgets

import ipywidgets as widgets
from IPython.display import display

# パラメータ設定用ウィジェット
symbol_widget = widgets.Dropdown(
    options=["BTC/USDT", "ETH/USDT", "SOL/USDT", "ADA/USDT", "DOT/USDT"],
    value="SOL/USDT",
    description="Symbol:"
)

timeframe_widget = widgets.Dropdown(
    options=["1m", "5m", "15m", "1h", "4h", "1d", "1w"],
    value="1d",
    description="Timeframe:"
)

limit_widget = widgets.IntSlider(
    value=1000,
    min=100,
    max=8760,
    step=100,
    description="期間(データ数):"
)

# ウィジェットを表示
print("📊 データ取得パラメータを設定してください:")
display(symbol_widget)
display(timeframe_widget)
display(limit_widget)

In [ ]:
# セル 4: データ取得実行 & 保存

import ccxt
from tenacity import retry, stop_after_attempt, wait_exponential

# KuCoin API設定
try:
    exchange = ccxt.kucoin({
        "apiKey": userdata.get("KuCoin_API_KEY"),
        "secret": userdata.get("KuCoin_API_SECRET"),
        "password": userdata.get("KuCoin_API_PASSPHRAS"),
        "enableRateLimit": True,
    })
    print("✅ KuCoin API接続完了")
except Exception as e:
    print(f"❌ API接続エラー: {e}")

# パラメータ取得
symbol = symbol_widget.value
timeframe = timeframe_widget.value
limit = limit_widget.value

print(f"\n📈 データ取得開始:")
print(f"  - Symbol: {symbol}")
print(f"  - Timeframe: {timeframe}")
print(f"  - Limit: {limit}")

# リトライ付きでデータ取得
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def fetch_ohlcv_with_retry(symbol, timeframe, limit):
    return exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

try:
    # OHLCVデータ取得
    ohlcv = fetch_ohlcv_with_retry(symbol, timeframe, limit)
    
    # DataFrameに変換
    data = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
    data = data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    data["timestamp"] = pd.to_datetime(data["timestamp"], unit="ms")
    data.set_index("timestamp", inplace=True)
    
    # 出力ディレクトリ作成
    output_dir = "/content/drive/MyDrive/kucoin_bot/data/raw/"
    os.makedirs(output_dir, exist_ok=True)
    
    # ファイル名生成
    safe_symbol = symbol.replace("/", "_")
    csv_filename = f"{safe_symbol}_{timeframe}_{limit}.csv"
    parquet_filename = f"{safe_symbol}_{timeframe}_{limit}.parquet"
    
    # CSV & Parquet で保存
    csv_path = os.path.join(output_dir, csv_filename)
    parquet_path = os.path.join(output_dir, parquet_filename)
    
    data.to_csv(csv_path)
    data.to_parquet(parquet_path)
    
    # 結果表示
    print(f"\n✅ データ取得完了！")
    print(f"📊 取得行数: {len(data)}行")
    print(f"📅 期間: {data.index[0]} ～ {data.index[-1]}")
    print(f"💾 保存先:")
    print(f"  - CSV: {csv_path}")
    print(f"  - Parquet: {parquet_path}")
    
    # データプレビュー
    print(f"\n📋 データプレビュー:")
    display(data.head())
    print(f"\n📈 基本統計:")
    display(data.describe())
    
except Exception as e:
    print(f"❌ データ取得エラー: {e}")